In [40]:
import numpy as np
import matplotlib.pyplot as plt

x_axis = np.load("bin_mids.npy")
wL = np.load("WL_sig_hist.npy")
wT = np.load("WL_bkg_hist.npy")

# pT 400 - 500

In [41]:
x_axis

array([0.01, 0.03, 0.05, 0.07, 0.09, 0.11, 0.13, 0.15, 0.17, 0.19, 0.21,
       0.23, 0.25, 0.27, 0.29, 0.31, 0.33, 0.35, 0.37, 0.39, 0.41, 0.43,
       0.45, 0.47, 0.49, 0.51, 0.53, 0.55, 0.57, 0.59, 0.61, 0.63, 0.65,
       0.67, 0.69, 0.71, 0.73, 0.75, 0.77, 0.79, 0.81, 0.83, 0.85, 0.87,
       0.89, 0.91, 0.93, 0.95, 0.97, 0.99])

In [42]:
print('Sum of wL : '+str(np.sum(wL)))
print('Sum of wT : '+str(np.sum(wT)))

Sum of wL : 118149
Sum of wT : 118369


## Definitions

In [46]:
from scipy import optimize
from numpy import exp, loadtxt, pi, sqrt
from scipy.integrate import quad

def gaussian(x, amp1, cen1, wid1, amp2, cen2, wid2):
    """1-d gaussian: gaussian(x, amp, cen, wid)"""
    return ((amp1/(sqrt(2*pi) * wid1)) * exp(-(x-cen1)**2/(2*wid1**2))+ 
            (amp2/(sqrt(2*pi) * wid2)) * exp(-(x-cen2)**2/(2*wid2**2)))

def background(x, amp1, cen1, wid1, amp2, cen2, wid2, amp3, cen3, wid3):
    return  ((amp1/(sqrt(2*pi) * wid1)) * exp(-(x-cen1)**2 / (2*wid1**2)) + 
             (amp2/(sqrt(2*pi) * wid2)) * exp(-(x-cen2)**2 / (2*wid2**2)) + 
             (amp3/(sqrt(2*pi) * wid3)) * exp(-(x-cen3)**2 / (2*wid3**2)))

def sig_func(x):
    return gaussian(x, params_WL[0], params_WL[1], params_WL[2], params_WL[3], params_WL[4], params_WL[5])

def bkg_func(x):
    return background(x, params_WT[0], params_WT[1], 
                           params_WT[2], params_WT[3], params_WT[4],
                           params_WT[5], params_WT[6], params_WT[7],
                           params_WT[8])

def fittot(x, amp1, amp2):
    return (amp1*(sig_func(x))+
            amp2*(bkg_func(x)))

def num_sig(amp): # Longitudinal W
    ans = 0
    for i in range(len(x_axis)):
        ans = ans + sig_func(x_axis[i])
    return ans*amp
    
def num_bkg(amp): # Transverse W
    ans = 0
    for i in range(len(x_axis)):
        ans = ans + bkg_func(x_axis[i])
    return ans*amp
    
def L_T_ratio(numL,numT):
    rat = numL/(numT+numL)
    return rat

## Longitudinal (Signal)

In [62]:

init_vals = [3000, 0.5, 0.3, 10000, 0.6, 0.02]

params_WL, params_covariance_WL = optimize.curve_fit(gaussian, x_axis, wL,
                                               p0=init_vals)

#def test_func(x, x0, x1, a, b, c, d):
#    return a * math.exp(-b * (x-x0)**2)+ c * math.exp(-d * (x-x1)**2)

#params, params_covariance = optimize.curve_fit(test_func, x_axis, wL, 
#                                               p0=[0.5, 0.6, 4000, 0.2, 12000, 0.05])

print(params_WL)
print("Number estimated from fit : "+str(num_sig(1))+", difference : "+str(np.sum(wL)-num_sig(1)))

[1.94566380e+03 5.56374808e-01 1.44731847e-01 4.21945841e+02
 6.26429756e-01 2.20061861e-02]
Number estimated from fit : 118269.70730007411, difference : -120.70730007410748


In [48]:
plt.scatter(x_axis, wL,label='data')
plt.plot(x_axis, gaussian(x_axis, params_WL[0], params_WL[1], 
                          params_WL[2], params_WL[3], params_WL[4], params_WL[5]),
         label='Fitted function')
plt.show()

## Transverse (Background) 

In [61]:
init_vals = [2000, 0.2, 0.01, 3000, 0.4, 0.02, 5000, 0.6, 0.02]
#init_vals = [3000, 0.2, 0.01, 4000, 0.6, 0.02, 4000, 0.6, 0.02, 4000, 0.6, 0.02]
#init_vals = [ 9000, 0.6, 0.02]

params_WT, params_covariance_WT = optimize.curve_fit(background, x_axis, wT,
                                               p0=init_vals)


print(params_WT)
print("Number estimated from fit : "+str(num_bkg(1))+", difference : "+str(np.sum(wT)-num_bkg(1)))

[2.47726147e+02 1.80887730e-01 4.29673997e-02 2.01062471e+03
 4.64707996e-01 1.56942631e-01 3.30058404e+02 6.19304892e-01
 5.24934741e-03]
Number estimated from fit : 121093.02454270134, difference : -2724.0245427013433


In [50]:
plt.scatter(x_axis, wT)

plt.plot(x_axis, background(x_axis, params_WT[0], params_WT[1], 
                           params_WT[2], params_WT[3], params_WT[4],
                           params_WT[5], params_WT[6], params_WT[7],
                           params_WT[8]),
         label='Fitted function')
plt.show()

# WZ Testing

In [51]:
wz = np.load("WZ_sig_hist.npy")


In [52]:
init_vals = [0.06, 0.11]

params_WZ, params_covariance_WZ = optimize.curve_fit(fittot, x_axis, wz,
                                               p0=init_vals)
print(params_WZ)

[0.06026028 0.11330313]


In [53]:
plt.scatter(x_axis, wz,label='data',color='grey')
plt.plot(x_axis, fittot(x_axis, params_WZ[0], params_WZ[1]),
         label='Fitted function')
plt.plot(x_axis, params_WZ[0]*sig_func(x_axis),
         label='WL')
plt.plot(x_axis, params_WZ[1]*bkg_func(x_axis),
         label='WT')
plt.legend()
plt.show()

In [54]:
longi_val = num_sig(params_WZ[0])
trans_val = num_bkg(params_WZ[1])
ltrat = L_T_ratio(longi_val,trans_val)

print(longi_val) #Something wrong too low values
print(trans_val)
print(ltrat)

7126.9660744846005
13720.218196323836
0.3418670829549022
